In [14]:
import os
import errno
import json
import sys
import requests
from cobrakbase.AbstractHandleClient import AbstractHandle as HandleService

In [15]:
class ShockException(Exception):
    pass

def download_file_from_kbase(
    token,
    file_id,
    file_path,
    is_handle_ref = 1,
    handle_url = "https://kbase.us/services/handle_service",
    shock_url = "https://kbase.us/services/shock-api",
):
    headers = {'Authorization': 'OAuth ' + token}  
    
    if is_handle_ref == 1:
        hs = HandleService(handle_url, token=token)
        handles = hs.hids_to_handles([file_id])
        file_id = handles[0]['id']
    
    try:
        os.makedirs(file_path)
    except OSError as exc:
        if exc.errno == errno.EEXIST and os.path.isdir(file_path):
            pass
        else:
            raise
    
    node_url = shock_url + '/node/' + file_id
    r = requests.get(node_url, headers=headers, allow_redirects=True)
    
    if not r.ok:
        errtxt = ('Error downloading file from shock ' +
          'node {}: ').format(file_id)
        try:
            err = json.loads(response.content)['error'][0]
        except Exception:
            # this means shock is down or not responding.
            log("Couldn't parse response error content from Shock: " +
                response.content)
            response.raise_for_status()
        raise ShockException(errtxt + str(err))
    
    resp_obj = r.json()
    size = resp_obj['data']['file']['size']
    if not size:
        raise ShockException('Node {} has no file'.format(shock_id))
    
    node_file_name = resp_obj['data']['file']['name']
    attributes = resp_obj['data']['attributes']
    if os.path.isdir(file_path):
        file_path = os.path.join(file_path, node_file_name)
    with open(file_path, 'wb') as fhandle:
        with requests.get(node_url + '?download_raw', stream=True,headers=headers, allow_redirects=True) as r:
            if not r.ok:
                errtxt = ('Error downloading file from shock ' +
                  'node {}: ').format(file_id)
                try:
                    err = json.loads(response.content)['error'][0]
                except Exception:
                    # this means shock is down or not responding.
                    log("Couldn't parse response error content from Shock: " +
                        response.content)
                    response.raise_for_status()
                raise ShockException(errtxt + str(err))
            for chunk in r.iter_content(1024):
                if not chunk:
                    break
                fhandle.write(chunk)
    return file_path

In [ ]:
print(download_file_from_kbase("<TOKEN>",file_id = "KBH_5023369",file_path = "/Users/chenry/"))